In [1]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")

from multi_group_simulation import MultiGroupSimulation
from stochastic_simulation import StochasticSimulation
from load_params import load_params

In [8]:
np.ones((2,2))

array([[1., 1.],
       [1., 1.]])

In [11]:
total_ug = (3533 + 8434) * 1.2 #assuming 20% higher enrolment in the fall



frac_vaccinated = 0.5
vaccinated_prev = 1
unvaccinated_prev = 10

vaccinated_test_freq = 2/7
unvaccinated_test_freq = 1/7



additional_classroom_interaction_matrix = 0.1 * np.ones((2,2))# for now


def gen_params(ug_ga_params, total_ug, frac_vaccinated, vaccinated_prev, unvaccinated_prev,
              additional_classroom_interaction_matrix, vaccinated_test_freq, unvaccinated_test_freq):
    ug_ga_params['exposed_infection_p'] = 0.0465 #calibrated transmission_p
    # Scaling of the interaction matrix is wrong, made up number of f/s -> f/s contact
    jan_11_interaction_matrix = 10 * np.array([[92/125, 1/44, 0],
                                              [3.5/125, 6.5/44, 0],
                                              [0,1/44,1/15]])

    jan_11_ug_ga_contacts = jan_11_interaction_matrix[0,0]

    fully_mixed_contacts_per_day = jan_11_ug_ga_contacts
    
    interaction_matrix = \
    [[fully_mixed_contacts_per_day * frac_vaccinated, fully_mixed_contacts_per_day * (1-frac_vaccinated)],
     [fully_mixed_contacts_per_day * frac_vaccinated, fully_mixed_contacts_per_day * (1-frac_vaccinated)]]
    
    vaccinated_params = ug_ga_params.copy()
    unvaccinated_params = ug_ga_params.copy()

    vaccinated_params['population_size'] = int(total_ug * frac_vaccinated)
    unvaccinated_params['population_size'] = int(total_ug * (1-frac_vaccinated))

    vaccinated_params['initial_ID_prevalence'] = vaccinated_prev /  int(total_ug * frac_vaccinated)
    unvaccinated_params['initial_ID_prevalence'] = unvaccinated_prev / int(total_ug * (1-frac_vaccinated))

    # split up contacts/day uniformly across vaccinated & unvaccinated populations
    # order is [vacc, unvacc]

    # vacc->vacc is fully_mixed * (vacc / total_pop)


    interaction_matrix = interaction_matrix + additional_classroom_interaction_matrix
    vaccinated_params['expected_contacts_per_day'] = interaction_matrix[0,0]
    unvaccinated_params['expected_contacts_per_day'] = interaction_matrix[1,1]

    vaccinated_params['test_population_fraction'] = vaccinated_test_freq
    unvaccinated_params['test_population_fraction'] = unvaccinated_test_freq


    params_list = [vaccinated_params, unvaccinated_params]
    group_names = ['vaccinated', 'unvaccinated']
    return params_list, interaction_matrix, group_names

def gen_sim(ug_ga_params, total_ug, frac_vaccinated, vaccinated_prev, unvaccinated_prev, 
              additional_classroom_interaction_matrix, vaccinated_test_freq, unvaccinated_test_freq):
    
    params_list, interaction_mtx, grp_names = gen_params(ug_ga_params, 
                                                            total_ug, 
                                                            frac_vaccinated, 
                                                            vaccinated_prev, 
                                                            unvaccinated_prev, 
                                                            additional_classroom_interaction_matrix, 
                                                            vaccinated_test_freq, 
                                                            unvaccinated_test_freq)

    return MultiGroupSimulation(params_list, interaction_mtx, grp_names)


sim = gen_sim(ug_ga_params, total_ug, frac_vaccinated, vaccinated_prev, unvaccinated_prev, 
              additional_classroom_interaction_matrix, vaccinated_test_freq, unvaccinated_test_freq)




In [12]:
_, sim_df = sim.run_new_trajectory(14)

In [13]:
sim_df

,S,QS,QI,R,E_0,E_1,E_2,E_3,E_4,E_5,...,SyID_severe_17,SyID_severe_18,SyID_severe_19,cumulative_mild,cumulative_severe,cumulative_outside_infections,severity_0,severity_1,severity_2,severity_3
0,14349,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0.975404,0.0180158,0.00658005
1,14346,0,2,0,0,0,1,0,0,0,...,0,0,0,2,3,0,2,2.92621,0.0540474,0.0197401
2,14338,6,4,0,1,0,0,0,0,0,...,0,0,0,4,6,1,4,5.85242,0.108095,0.0394803
3,14335,4,4,0,0,3,0,1,0,0,...,0,0,0,5,7,1,5,6.82783,0.126111,0.0460603
4,14331,4,5,0,4,1,2,0,1,0,...,0,0,0,6,7,2,6,6.82783,0.126111,0.0460603
5,14323,7,9,0,2,3,0,1,0,0,...,0,0,0,7,9,3,7,8.77864,0.162142,0.0592204
6,14315,10,13,1,3,0,0,2,0,0,...,0,0,0,9,14,4,9,13.6557,0.252221,0.0921207
7,14303,15,12,5,3,0,1,0,0,0,...,0,0,0,11,15,4,11,14.6311,0.270237,0.0987007
8,14296,17,15,7,2,0,1,0,0,0,...,0,1,0,16,17,4,16,16.5819,0.306269,0.111861
9,14277,26,21,8,0,2,0,2,0,0,...,0,0,0,20,22,4,20,21.4589,0.396348,0.144761


In [5]:
ug_ga_params

{'ID_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>>,
 'SyID_mild_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>>,
 'SyID_severe_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>>,
 'cases_isolated_per_contact': 1.329,
 'cases_quarantined_per_contact': 3.304,
 'contact_trace_testing_frac': 1,
 'contact_tracing_delay': 1,
 'daily_outside_infection_p': 1.42e-05,
 'days_between_tests': 1,
 'expected_contacts_per_day': 0.736,
 'exposed_infection_p': 0.017,
 'exposed_time_function': <function analysis_helpers.poisson_waiting_function.<locals>.<lambda>>,
 'initial_E_count': 0,
 'initial_ID_count': 0,
 'initial_ID_prevalence': 0,
 'initial_SyID_mild_count': 0,
 'initial_SyID_severe_count': 0,
 'initial_pre_ID_count': 0,
 'max_time_ID': 8,
 'max_time_SyID_mild': 20,
 'max_time_SyID_severe': 20,
 'max_time_exposed': 7,
 'max_time_pre_ID': 4,
 'mild_severity_levels': 1,
 'mild_sympto

In [2]:
# Fall Semester Group Config

# Loading group params
base_directory = '../src/simulations_v2/params/jan4calibration/'
# base_directory = '../src/simulations_v2/params/jan_12_send_to_mike/'

ug_ga_params = load_params(base_directory + 'group_1_students_post_movein_private.yaml')[1]
ug_other_params = load_params(base_directory + 'group_2_students_post_movein_private.yaml')[1]
gs_params = load_params(base_directory + 'group_3_students_post_movein_private.yaml')[1]
fs_params = load_params(base_directory + 'faculty_staff_post_movein_private.yaml')[1]
#params_list = [ug_ga_params.copy(), ug_other_params.copy(), gs_params.copy(), fs_params.copy()]
params_list = [ug_ga_params.copy(), ug_other_params.copy(), gs_params.copy()]

In [3]:
# Scaling of the interaction matrix is wrong, made up number of f/s -> f/s contact
interaction_matrix = 10 * np.array([[92/125, 1/44, 0],
                              [3.5/125, 6.5/44, 0],
                              [0,1/44,1/15]])

# adding population size
params_list[0]['population_size'] = 3533
params_list[1]['population_size'] = 8434
params_list[2]['population_size'] = 6202
#params_list[3]['population_size'] = 10000

for idx in range(3):
    params_list[idx]['daily_outside_infection_p'] *= 2
    params_list[idx]['expected_contacts_per_day'] = interaction_matrix[idx][idx]
#     params_list[idx]['test_protocol_QFNR'] = 1 - (0.75 * 0.95)

# Initially 12.4 free + infectious individuals (UG only)
UG_0_initial_cases = 5.69
UG_1_initial_cases = 13.15
UG_2_initial_cases = 6.52

params_list[0]['initial_ID_prevalence'] = UG_0_initial_cases / params_list[0]['population_size']
params_list[1]['initial_ID_prevalence'] = UG_1_initial_cases / params_list[1]['population_size']
params_list[2]['initial_ID_prevalence'] = UG_2_initial_cases / params_list[2]['population_size']

group_names = ['UG (Greek, Athlete)', 'UG (other)', 'GS']#, 'Faculty/Staff']